# Setup

In [1]:
import os
import logging as log

from lisa.utils import setup_logging
setup_logging(level=log.ERROR)

from lisa.trace import Trace
from lisa.wa import WAOutput
from lisa.stats import Stats
from lisa.datautils import series_mean
from pandas import DataFrame
import pandas as pd
import scipy as sp
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import holoviews as hv
from holoviews import opts
from bokeh.themes import built_in_themes
from tabulate import tabulate

from holoviews.operation.datashader import datashade, rasterize
from holoviews.operation import decimate

from wp.notebook import WorkloadNotebookAnalysis, setup_notebook

setup_notebook()

## Runs

In [2]:
jank = WorkloadNotebookAnalysis('/data_sda/honxia02/lisa/', ['jankbench_mainfridge_10_2609', 'jankbench_sumfridge_10_2609'])
jank.show()

,test_name,rep,metric,value,unit,iteration,workload,id,wa_path,kernel,tag
0,Bitmap Upload Test,0.0,mean,10.362638,,1,jankbench,wk1,jankbench_mainfridge_10_2609,5.18.0-armpower-00029-gc35255a0e739,mainfridge
1,Bitmap Upload Test,0.0,std_dev,2.545276,,1,jankbench,wk1,jankbench_mainfridge_10_2609,5.18.0-armpower-00029-gc35255a0e739,mainfridge
2,Bitmap Upload Test,0.0,count_jank,7.000000,,1,jankbench,wk1,jankbench_mainfridge_10_2609,5.18.0-armpower-00029-gc35255a0e739,mainfridge
3,Bitmap Upload Test,0.0,jank_p,0.254823,,1,jankbench,wk1,jankbench_mainfridge_10_2609,5.18.0-armpower-00029-gc35255a0e739,mainfridge
4,Edit Text Input,0.0,mean,6.776690,,1,jankbench,wk1,jankbench_mainfridge_10_2609,5.18.0-armpower-00029-gc35255a0e739,mainfridge
...,...,...,...,...,...,...,...,...,...,...,...
635,Overdraw Test,0.0,jank_p,0.073287,,10,jankbench,wk1,jankbench_sumfridge_10_2609,5.18.0-armpower-00037-gfdf7806135b3,sumfridge
636,Shadow Grid Fling,0.0,mean,6.087260,,10,jankbench,wk1,jankbench_sumfridge_10_2609,5.18.0-armpower-00037-gfdf7806135b3,sumfridge
637,Shadow Grid Fling,0.0,std_dev,2.561411,,10,jankbench,wk1,jankbench_sumfridge_10_2609,5.18.0-armpower-00037-gfdf7806135b3,sumfridge
638,Shadow Grid Fling,0.0,count_jank,3.000000,,10,jankbench,wk1,jankbench_sumfridge_10_2609,5.18.0-armpower-00037-gfdf7806135b3,sumfridge


benchmark_dirs: ['jankbench_mainfridge_10_2609', 'jankbench_sumfridge_10_2609']
tags: ['mainfridge', 'sumfridge']
kernels: ['5.18.0-armpower-00029-gc35255a0e739', '5.18.0-armpower-00037-gfdf7806135b3']


# Benchmark scores

In [3]:
jank.plot.jankbench_jank_percentage_line()

:NdOverlay   [tag]
   :Curve   [iteration]   (perc)

In [4]:
jank.plot.jankbench_jank_percentage_bar()

+--------+------------+-----------------+-------+-----------+
| metric |    tag     |    variable     | value | perc_diff |
+--------+------------+-----------------+-------+-----------+
| gmean  | mainfridge | jank_percentage |  1.1  |   0.0%    |
| gmean  | sumfridge  | jank_percentage |  0.5  |  -53.92%  |
+--------+------------+-----------------+-------+-----------+


## Jankbench metrics per iteration

## Bar plot - jank percentage

In [5]:
jank.plot.jankbench_jank_percentage_metric_bar()

+--------+------------+--------------------------+-------+-----------+
| metric |    tag     |        test_name         | value | perc_diff |
+--------+------------+--------------------------+-------+-----------+
| jank_p | mainfridge |    Bitmap Upload Test    |  0.5  |   0.0%    |
| jank_p | mainfridge |     Edit Text Input      |  2.9  |   0.0%    |
| jank_p | mainfridge | High-hitrate text render |  1.5  |   0.0%    |
| jank_p | mainfridge |  Image List View Fling   |  0.9  |   0.0%    |
| jank_p | mainfridge |     List View Fling      |  1.1  |   0.0%    |
| jank_p | mainfridge | Low-hitrate text render  |  0.9  |   0.0%    |
| jank_p | mainfridge |      Overdraw Test       |  0.3  |   0.0%    |
| jank_p | mainfridge |    Shadow Grid Fling     |  0.4  |   0.0%    |
| jank_p | sumfridge  |    Bitmap Upload Test    |  0.2  |  -63.8%   |
| jank_p | sumfridge  |     Edit Text Input      |  2.2  |  -24.01%  |
| jank_p | sumfridge  | High-hitrate text render |  0.2  |  -86.34%  |
| jank

## Bar plot - mean duration

# Overutilized

## Line plot

In [6]:
jank.plot.overutilized_line()

+--------------+------------+------+------------+------------+
|    metric    |    tag     | time | total_time | percentage |
+--------------+------------+------+------------+------------+
| overutilized | mainfridge | 8.51 |   254.32   |    3.35    |
| overutilized | sumfridge  | 4.37 |   256.08   |    1.71    |
+--------------+------------+------+------------+------------+


# Power usage

## Bar plot

In [7]:
jank.plot.power_meter_bar()

+------------+--------+------------+-------+-----------+
|  channel   | metric |    tag     | value | perc_diff |
+------------+--------+------------+-------+-----------+
|    CPU     | gmean  | mainfridge | 166.1 |   0.0%    |
|  CPU-Big   | gmean  | mainfridge | 55.1  |   0.0%    |
| CPU-Little | gmean  | mainfridge | 91.7  |   0.0%    |
|  CPU-Mid   | gmean  | mainfridge | 19.2  |   0.0%    |
|    GPU     | gmean  | mainfridge | 33.0  |   0.0%    |
|   Total    | gmean  | mainfridge | 199.1 |   0.0%    |
|    CPU     | gmean  | sumfridge  | 161.3 |  -2.85%   |
|  CPU-Big   | gmean  | sumfridge  | 52.9  |  -3.97%   |
| CPU-Little | gmean  | sumfridge  | 86.7  |  -5.39%   |
|  CPU-Mid   | gmean  | sumfridge  | 21.6  |  12.63%   |
|    GPU     | gmean  | sumfridge  | 34.3  |   3.84%   |
|   Total    | gmean  | sumfridge  | 195.7 |  -1.74%   |
+------------+--------+------------+-------+-----------+
